In [3]:
import argparse
import os
import numpy as np
import torch
import cv2, glob
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from modeling.sync_batchnorm.replicate import patch_replication_callback
from modeling.deeplab import *

from dataloaders.utils import decode_segmap
MI = np.load('meanimage.npy')

numClass={'pascal':6,
'coco':21,
'cityscapes':19}
classes = ['drusen', 'hemorrhage', 'exudate', 'scar', 'others']
for cid, c in enumerate(classes):
    if not os.path.isdir('result/'+c):
        os.mkdir('result/' + c)

# parser = argparse.ArgumentParser(description="PyTorch DeeplabV3Plus Training")

# parser.add_argument('--backbone', type=str, default='resnet',choices=['resnet', 'xception', 'drn', 'mobilenet'],help='backbone name (default: resnet)')
# parser.add_argument('--out-stride', type=int, default=16,help='network output stride (default: 8)')
# parser.add_argument('--dataset', type=str, default='cityscapes', choices=['pascal', 'coco', 'cityscapes'],help='dataset name (default: pascal)')
# parser.add_argument('--sync-bn', type=bool, default=None,help='whether to use sync bn (default: auto)')
# parser.add_argument('--freeze-bn', type=bool, default=False,help='whether to freeze bn parameters (default: False)')
# parser.add_argument('--weightPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--imgPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--outPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--no-cuda', action='store_true', default=False, help='disables CUDA training')
# args = parser.parse_args()
cuda = torch.cuda.is_available()
cuda = True
loc = 'cuda' if cuda else 'cpu'
nclass = numClass['pascal']
model = DeepLab(num_classes=nclass, backbone='resnet', output_stride=8, sync_bn=None, freeze_bn=False)
weight_dict=torch.load('run/pascal/task48022/model_best.pth.tar', map_location=loc)
if cuda:
    model = torch.nn.DataParallel(model, device_ids=[0])
    patch_replication_callback(model)
    model = model.cuda()
    
    model.module.load_state_dict(weight_dict['state_dict'])
else:
    model.load_state_dict(weight_dict['state_dict'])
model.eval()


filenames = glob.glob('Validation-400-images/*.jpg')
kernel = np.ones((5,5),np.uint8)
kernel2 = np.ones((11, 11),np.uint8)
for imgPath in filenames:
    fn = os.path.basename(imgPath)[:-4]
    outPath = 'result/' + fn +'.png'

    image = cv2.imread(imgPath)
    oriDim = image.shape
    image = cv2.resize(image, dsize=(480,480)) - cv2.resize(MI, (480,480))
#     image = cv2.resize(image, dsize=(513,513)) 
    image = image.astype(np.float32) / 255.
    image = image[:, :, ::-1]
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    
    
    
    for i in range(3):
        image[:, :, i] = image[:, :, i] - means[i]
        image[:, :, i] = image[:, :, i] / stds[i]

    image = torch.from_numpy(image.transpose((2, 0, 1)).astype(np.float32)).float().unsqueeze(0)

    if cuda:
        image = image.cuda()
        
    with torch.no_grad():
        output = model(image)
        output = output.data.cpu().numpy()
        prediction = np.argmax(output, axis=1)[0]
        ps=[]
        for cid, c in enumerate(classes):
            mask = np.zeros((prediction.shape[0], prediction.shape[1]), np.uint8) +255
            mask[prediction == cid+1] = 0
#             mask = output[0, cid+1]
#             mask[mask>0.3] = 255
#             mask[mask<=0.3] = 0
            mask = mask.astype(np.uint8)
            mask = cv2.morphologyEx(255-mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel2)
            hole=mask.copy()

            cv2.floodFill(hole,None,(0,0),255) # 找到洞孔
            hole=cv2.bitwise_not(hole)
            mask=255-cv2.bitwise_or(mask,hole)

            mask = cv2.resize(mask,dsize=(oriDim[1],oriDim[0]), interpolation=cv2.INTER_NEAREST)
            ps.append(np.mean(prediction == cid+1))
            cv2.imwrite('result/' + c + '/' + fn+'.png', mask)

        segmap = decode_segmap(prediction, dataset='pascal')
        segmap = (segmap*255).astype(np.uint8)
        segmap = cv2.resize(segmap,dsize=(oriDim[1],oriDim[0]))
        segmap = segmap[:, :, ::-1]
        cv2.imwrite(outPath, segmap)
    print('Done inference '+fn,'percentage:', ps)
exit(1)

Done inference V0366 percentage: [0.005889756944444445, 0.0, 0.0, 0.0, 0.0]
Done inference V0213 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0333 percentage: [0.1398220486111111, 0.0, 0.0, 0.0, 0.0]
Done inference V0108 percentage: [0.0010112847222222222, 0.0, 0.011180555555555555, 0.00046875, 0.0]
Done inference V0264 percentage: [0.0, 0.0, 0.015334201388888889, 0.0, 0.0]
Done inference V0323 percentage: [0.0, 0.014231770833333334, 0.0002300347222222222, 0.0623046875, 0.0]
Done inference V0347 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0064 percentage: [0.0021006944444444445, 0.0, 0.0, 0.0, 0.0]
Done inference V0389 percentage: [0.005078125, 0.0, 0.01884548611111111, 0.0, 0.0]
Done inference V0381 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0306 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0307 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0188 percentage: [0.004453125, 0.0, 0.007486979166666667, 0.0005381944444444444, 0.0]
Done inferenc

Done inference V0157 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0354 percentage: [0.0032421875, 0.0, 0.12381944444444444, 0.011028645833333333, 0.0]
Done inference V0019 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0104 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0119 percentage: [0.0, 0.019887152777777778, 0.0, 0.0, 0.0]
Done inference V0371 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0010 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0082 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0194 percentage: [0.13907986111111112, 0.0, 0.0, 0.0, 0.0]
Done inference V0332 percentage: [0.005724826388888889, 0.0, 0.0, 0.0, 0.0]
Done inference V0230 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0059 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0278 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0134 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0327 percentage: [0.0052517361111111115, 0.0, 0.0, 0.0, 0.0]
Done 

Done inference V0240 percentage: [0.00046440972222222223, 0.0, 0.0, 0.0, 0.0]
Done inference V0207 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0255 percentage: [0.0004947916666666667, 0.0, 0.0, 0.0, 0.0]
Done inference V0116 percentage: [0.021966145833333332, 0.0, 0.0, 0.00024305555555555555, 0.0]
Done inference V0179 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0301 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0337 percentage: [0.0, 0.017204861111111112, 0.08154513888888888, 0.0686328125, 0.0]
Done inference V0340 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0375 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0071 percentage: [0.0012717013888888888, 0.0, 0.004674479166666667, 0.0, 0.0]
Done inference V0079 percentage: [0.0017013888888888888, 0.0, 0.0, 0.0, 0.0]
Done inference V0395 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0236 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0009 percentage: [0.0, 0.05116753472222222, 0.850

Done inference V0041 percentage: [0.00017361111111111112, 0.0, 0.0, 0.0, 0.0]
Done inference V0220 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0359 percentage: [0.009700520833333334, 0.0, 0.0, 0.0, 0.0]
Done inference V0146 percentage: [0.0, 0.0, 0.10295138888888888, 0.0, 0.0]
Done inference V0017 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0233 percentage: [0.0027430555555555554, 0.0, 0.0, 0.0, 0.0]
Done inference V0107 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0212 percentage: [0.001072048611111111, 0.0, 0.0, 0.0, 0.0]
Done inference V0102 percentage: [0.0, 0.0, 0.0, 0.0, 0.0]
Done inference V0013 percentage: [0.004539930555555556, 0.0, 0.029071180555555555, 0.0, 0.0]
Done inference V0127 percentage: [0.0, 0.0, 0.0, 0.0, 0.0023003472222222223]
Done inference V0336 percentage: [0.000390625, 0.0, 0.0, 0.0, 0.0]
Done inference V0163 percentage: [0.0, 0.0, 0.0014800347222222222, 0.0, 0.0]
Done inference V0061 percentage: [0.0020442708333333333, 0.0, 0.0005